In [1]:
from edgar_functions import *
from scorecard_naming import (
    Balance_sheet_mapping,
    Income_statement_mapping,
    Cash_flow_mapping,
)

ticker = "wsm"

In [2]:
income_sheet = pd.read_csv(
    "simplified_statements/wsm/income_statement_annual.csv", index_col=0
)
balance_sheet = pd.read_csv(
    "simplified_statements/wsm/balance_sheet_annual.csv", index_col=0
)
cash_flow_sheet = pd.read_csv(
    "simplified_statements/wsm/cash_flow_statement_annual.csv", index_col=0
)
# income_sheet_quarterly = form_statement_for_all_available_quarters(ticker, "income_statement")
# balance_sheet_quarterly = form_statement_for_all_available_quarters(ticker, "balance_sheet")
# cash_flow_sheet_quarterly = form_statement_for_all_available_quarters(ticker, "cash_flow_statement")

In [23]:
def create_scorecard(income_sheet, balance_sheet, cash_flow_sheet):
    scorecard = pd.DataFrame()
    scorecard["Net Revenue"] = income_sheet["Net Revenue"]
    scorecard["COGS"] = income_sheet["Cost of Goods Sold"]
    scorecard["Gross Profit"] = income_sheet["Gross Profit"]
    scorecard["SG&A"] = income_sheet["SG&A"]
    scorecard["Operating Expenses"] = income_sheet["SG&A"]
    scorecard["Operating Income"] = income_sheet["Operating Income"]
    scorecard["Interest Expense"] = income_sheet["Interest Expense"]
    scorecard["EBT"] = income_sheet["EBT"]
    scorecard["Taxes"] = income_sheet["Taxes"]
    scorecard["Tax Rate"] = scorecard["Taxes"] / scorecard["EBT"]
    scorecard["Net Income"] = income_sheet["Net income"]
    scorecard["EPS"] = income_sheet["EPS"]
    scorecard["EPS Diluted"] = income_sheet["EPS Diluted"]
    scorecard["EBITDA"] = (
        income_sheet["EBT"]
        + income_sheet["Interest Expense"]
        + cash_flow_sheet["Depreciation and Amortization"]
    )
    scorecard["Dividend per Share(diluted)"] = (
        -cash_flow_sheet["Payment of Dividends"]
    ) / income_sheet["Weighted Average Shares Diluted"]
    scorecard["Dividend Payout Ratio"] = (
        scorecard["Dividend per Share(diluted)"] / scorecard["EPS Diluted"]
    )
    scorecard["Cash"] = balance_sheet["Cash"]
    scorecard["Accounts Receivable"] = balance_sheet["Accounts Receivable"]
    scorecard["Inventory"] = balance_sheet["Inventory"]
    scorecard["Other Current Assets"] = balance_sheet["Other Current Assets"]
    scorecard["Total Current Assets"] = balance_sheet["Total Current Assets"]
    scorecard["Property, Plant & Equipment, Net"] = balance_sheet[
        "Property, Plant, and Equipment"
    ]
    scorecard["Goodwill"] = balance_sheet["Goodwill"]
    scorecard["Total Assets"] = balance_sheet["Total Assets"]
    scorecard["Accrued Expenses"] = balance_sheet["Accrued Expenses"]
    scorecard["Deferred Revenue"] = balance_sheet["Deferred Revenue"]
    scorecard["Income Tax Payable"] = balance_sheet["Income Taxes Payable"]
    scorecard["Operating Lease Liability"] = balance_sheet["Operating Lease Liability"]
    scorecard["Other Current Liabilities"] = balance_sheet["Other Current Liabilities"]
    scorecard["Accounts Payable"] = balance_sheet["Accounts Payable"]
    scorecard["Total Current Liabilities"] = balance_sheet["Total Current Liabilities"]
    scorecard["Deffered Lease Incentives"] = balance_sheet["Deffered Lease Incentive"]
    scorecard["Long Term Lease Liability"] = balance_sheet["Long Term Lease Liability"]
    scorecard["Other Long Term Liabilities"] = balance_sheet[
        "Other Long Term Liabilities"
    ]
    scorecard["Total Liabilities"] = balance_sheet["Total Liabilities"]
    scorecard["Equity"] = balance_sheet["Total Stockholders Equity"]
    scorecard["Total Liabilities and Equity"] = balance_sheet[
        "Total Liabilities and Stockholders Equity"
    ]
    scorecard["Book Value per Share"] = (
        balance_sheet["Total Stockholders Equity"]
        / income_sheet["Weighted Average Shares Diluted"]
    )
    scorecard["Operating Cash Flow"] = cash_flow_sheet[
        "Net Cash Provided by Operating Activities"
    ]
    scorecard["OCF/NI"] = scorecard["Operating Cash Flow"] / scorecard["Net Income"]
    scorecard["Depreciation & Amortization"] = cash_flow_sheet[
        "Depreciation and Amortization"
    ]
    scorecard["Capital Expenditure"] = -cash_flow_sheet[
        "Purchase of Property, Plant, and Equipment"
    ]
    scorecard["Free Cash Flow"] = (
        scorecard["Operating Cash Flow"] + scorecard["Capital Expenditure"]
    )
    scorecard["Capex/Depreciation"] = (
        scorecard["Capital Expenditure"] / scorecard["Depreciation & Amortization"]
    )
    scorecard["Dividends"] = -cash_flow_sheet["Payment of Dividends"]
    scorecard["Stock Repurchase"] = -cash_flow_sheet["Repurchases of Common Stock"]
    scorecard["ROA"] = scorecard["Net Income"] / scorecard["Total Assets"]
    scorecard["ROE"] = scorecard["Net Income"] / scorecard["Equity"]
    scorecard["Profit Margin"] = scorecard["Net Income"] / scorecard["Net Revenue"]
    scorecard["Asset Turnover"] = scorecard["Net Revenue"] / scorecard["Total Assets"]
    scorecard["Current Ratio"] = (
        scorecard["Total Current Assets"] / scorecard["Total Current Liabilities"]
    )
    scorecard["Equity Multiplier"] = scorecard["Total Assets"] / scorecard["Equity"]
    scorecard["Net Working Capital"] = (
        scorecard["Total Current Assets"] - scorecard["Total Current Liabilities"]
    )
    scorecard["Debt to Equity Ratio"] = (
        scorecard["Total Liabilities"] / scorecard["Equity"]
    )
    scorecard["Days of Sales Outstanding"] = (
        scorecard["Accounts Receivable"] / scorecard["Net Revenue"]
    ) * 365
    scorecard["Days of Inventory on Hand"] = (
        scorecard["Inventory"] / scorecard["COGS"]
    ) * 365
    scorecard["Payables Period"] = (
        scorecard["Accounts Payable"] / scorecard["COGS"]
    ) * 365
    scorecard["Cash Cycle"] = (
        scorecard["Days of Sales Outstanding"]
        + scorecard["Days of Inventory on Hand"]
        - scorecard["Payables Period"]
    )
    return scorecard

In [32]:
scorecard = create_scorecard(income_sheet, balance_sheet, cash_flow_sheet)

In [33]:
scorecard.drop("2010-01-31", axis=0, inplace=True)
scorecard.drop("2011-01-30", axis=0, inplace=True)

In [35]:
scorecard.T

,2023-01-29,2022-01-30,2021-01-31,2020-02-02,2019-02-03,2018-01-28,2017-01-29,2016-01-31,2015-02-01,2014-02-02,2013-02-03,2012-01-29
Net Revenue,8.674417e+06,8.245936e+06,6.783189e+06,5.898008e+06,5.671593e+06,5.292359e+06,5.083812e+06,4.976090e+06,4.698719e+06,4.387889e+06,4.042870e+06,3.720895e+06
COGS,4.996684e+06,4.613973e+06,4.146920e+06,3.758916e+06,3.570580e+06,3.360648e+06,3.200502e+06,3.131876e+06,2.898215e+06,2.683673e+06,2.450394e+06,2.261039e+06
Gross Profit,3.677733e+06,3.631963e+06,2.636269e+06,2.139092e+06,2.101013e+06,1.931711e+06,1.883310e+06,1.844214e+06,1.800504e+06,1.704216e+06,1.592476e+06,1.459856e+06
SG&A,2.179311e+06,2.178847e+06,1.725572e+06,1.673218e+06,1.665060e+06,1.477900e+06,1.410711e+06,1.355580e+06,1.298239e+06,1.252118e+06,1.183313e+06,1.078124e+06
Operating Expenses,2.179311e+06,2.178847e+06,1.725572e+06,1.673218e+06,1.665060e+06,1.477900e+06,1.410711e+06,1.355580e+06,1.298239e+06,1.252118e+06,1.183313e+06,1.078124e+06
Operating Income,1.498422e+06,1.453116e+06,9.106970e+05,4.658740e+05,4.359530e+05,4.538110e+05,4.725990e+05,4.886340e+05,5.022650e+05,4.520980e+05,4.091630e+05,3.817320e+05
Interest Expense,-2.260000e+03,1.865000e+03,1.623100e+04,8.853000e+03,6.706000e+03,1.372000e+03,6.880000e+02,6.270000e+02,6.200000e+01,-5.840000e+02,-7.930000e+02,-9.800000e+01
EBT,1.500682e+06,1.451251e+06,8.944660e+05,4.570210e+05,4.292470e+05,4.524390e+05,4.719110e+05,4.880070e+05,5.022030e+05,4.526820e+05,4.099560e+05,3.818300e+05
Taxes,3.727780e+05,3.249140e+05,2.137520e+05,1.009590e+05,9.556300e+04,1.928940e+05,1.665240e+05,1.779390e+05,1.933490e+05,1.737800e+05,1.532260e+05,1.448990e+05
Tax Rate,2.484057e-01,2.238855e-01,2.389716e-01,2.209067e-01,2.226294e-01,4.263426e-01,3.528716e-01,3.646239e-01,3.850017e-01,3.838898e-01,3.737621e-01,3.794856e-01


In [8]:
cash_flow_sheet.T

,2023-01-29,2022-01-30,2021-01-31,2020-02-02,2019-02-03,2018-01-28,2017-01-29,2016-01-31,2015-02-01,2014-02-02,2013-02-03,2012-01-29,2011-01-30,2010-01-31
Net Earnings,1127904.0,1126337.0,680714.0,356062.0,333684.0,259545.0,305387.0,310068.0,308854.0,278902.0,256730.0,236931.0,200227.0,77442.0
Depreciation and Amortization,214153.0,196087.0,188655.0,187759.0,188808.0,183077.0,173195.0,167760.0,162273.0,149795.0,134453.0,130553.0,144630.0,151796.0
Loss on Disposal/Impairment,25116.0,1015.0,32365.0,1755.0,10209.0,1889.0,3806.0,4339.0,2410.0,2764.0,8388.0,2880.0,0.0,0.0
Amoritization of Deferred Lease Incentive,-3019.0,-4282.0,-5783.0,-7714.0,-26199.0,-25372.0,-25212.0,-24721.0,-24419.0,-25382.0,-26694.0,-27547.0,-37115.0,-36799.0
Non Cash Lease Expense,231350.0,216888.0,216368.0,215810.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Deferred Income Taxes,-23823.0,2535.0,-13061.0,-2557.0,23639.0,63381.0,7114.0,-7436.0,-248.0,-28344.0,-9029.0,14210.0,23566.0,-23595.0
Stock Based Compensation Expense,90268.0,95240.0,73185.0,64163.0,59802.0,42988.0,51116.0,41357.0,44632.0,38788.0,31042.0,24336.0,26630.0,24989.0
Other Operating Activities,680.0,288.0,-264.0,-26.0,-579.0,-135.0,-423.0,149.0,595.0,0.0,0.0,17.0,0.0,0.0
Changes in Accounts Receivable,15687.0,11896.0,-31503.0,-5034.0,-15329.0,149.0,-9794.0,-12849.0,-9366.0,786.0,-16408.0,-4763.0,3477.0,-6620.0
Changes in Inventory,-208908.0,-239981.0,99144.0,24219.0,-70331.0,-80235.0,4493.0,-92647.0,-76964.0,-174664.0,-85981.0,-34853.0,-46464.0,108332.0


In [7]:
balance_sheet.T

,2023-01-29,2022-01-30,2021-01-31,2020-02-02,2019-02-03,2018-01-28,2017-01-29,2016-01-31,2015-02-01,2014-02-02,2013-02-03,2012-01-29,2011-01-30
Cash,367344.0,850338.0,1200337.0,432162.0,338954.0,390136.0,213713.0,193647.0,222927.0,330121.0,424555.0,502757.0,6.284030e+08
Accounts Receivable,115685.0,131683.0,143728.0,111737.0,107102.0,90119.0,88803.0,79304.0,67465.0,60330.0,62985.0,45961.0,4.156500e+07
Inventory,1456123.0,1246372.0,1006299.0,1100544.0,1124992.0,1061593.0,977505.0,978138.0,887701.0,813160.0,640024.0,553461.0,5.133810e+08
Prepaid Expenses,64961.0,69252.0,93822.0,90426.0,101356.0,62204.0,52882.0,44654.0,36265.0,35309.0,26339.0,24188.0,2.112000e+07
Other Current Assets,31967.0,26249.0,22894.0,20766.0,21939.0,11876.0,10652.0,11438.0,13005.0,10852.0,9819.0,9229.0,8.176000e+06
Total Current Assets,2036080.0,2323894.0,2467080.0,1755635.0,1694343.0,1636445.0,1367180.0,1336100.0,1391923.0,1419103.0,1316772.0,1276366.0,1.347594e+09
"Property, Plant, and Equipment",1065381.0,920773.0,873894.0,929038.0,929635.0,932283.0,923283.0,886813.0,883012.0,849293.0,812037.0,734672.0,7.305560e+08
Operating Lease Right of Use Asset,1286452.0,1132764.0,1086009.0,1166383.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
Deffered Income Taxes,81389.0,56585.0,61854.0,47977.0,44055.0,67306.0,135238.0,141784.0,4265.0,13824.0,12398.0,12382.0,3.264600e+07
Goodwill,77307.0,85354.0,85446.0,85343.0,85382.0,18838.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00


In [3]:
income_sheet

,Net Revenue,Cost of Goods Sold,Gross Profit,SG&A,Operating Income,Interest Expense,EBT,Taxes,Net income,EPS,EPS Diluted,Weighted Average Shares Outstanding,Weighted Average Shares Diluted
2023-01-29,8674417.0,4996684.0,3677733.0,2179311.0,1498422.0,-2260.0,1500682.0,372778.0,1127904.0,16.58,16.32,68021.0,69100.0
2022-01-30,8245936.0,4613973.0,3631963.0,2178847.0,1453116.0,1865.0,1451251.0,324914.0,1126337.0,15.17,14.75,74272.0,76354.0
2021-01-31,6783189.0,4146920.0,2636269.0,1725572.0,910697.0,16231.0,894466.0,213752.0,680714.0,8.81,8.61,77260.0,79055.0
2020-02-02,5898008.0,3758916.0,2139092.0,1673218.0,465874.0,8853.0,457021.0,100959.0,356062.0,4.56,4.49,78108.0,79225.0
2019-02-03,5671593.0,3570580.0,2101013.0,1665060.0,435953.0,6706.0,429247.0,95563.0,333684.0,4.10,4.05,81420.0,82340.0
2018-01-28,5292359.0,3360648.0,1931711.0,1477900.0,453811.0,1372.0,452439.0,192894.0,259545.0,3.03,3.02,85592.0,86080.0
2017-01-29,5083812.0,3200502.0,1883310.0,1410711.0,472599.0,688.0,471911.0,166524.0,305387.0,3.45,3.41,88594.0,89462.0
2016-01-31,4976090.0,3131876.0,1844214.0,1355580.0,488634.0,627.0,488007.0,177939.0,310068.0,3.42,3.37,90787.0,92102.0
2015-02-01,4698719.0,2898215.0,1800504.0,1298239.0,502265.0,62.0,502203.0,193349.0,308854.0,3.30,3.24,93634.0,95200.0
2014-02-02,4387889.0,2683673.0,1704216.0,1252118.0,452098.0,-584.0,452682.0,173780.0,278902.0,2.89,2.82,96669.0,98765.0
